# Memory

Langgraph is designed to graph all events that occur within the graph. This section considers options for setting up and using the graph's memory.

In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

## Get state

You can obtain a state of the graph compiled with the checkpointer using `get_state` method.

---

The following cell creates a graph with a checkpointer and invokes it.

In [4]:
graph = (
    StateGraph(dict)
    .add_edge(START, END)
    .compile(checkpointer=InMemorySaver())
)


config = {"configurable": {"thread_id": 1}}
graph.invoke(dict(val=10), config=config)

{'val': 10}

The following cell shows the output of the `get_config` method.

In [5]:
graph.get_state(config)

StateSnapshot(values={'val': 10}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f9e32-8aaa-6205-8000-33f823d969bd'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-01-25T11:44:08.364876+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f9e32-8aa3-66b2-bfff-aa39a0d9d120'}}, tasks=(), interrupts=())

## Get history

You can retrieve the history of the updates of the state by invoking the `get_state_history` method of the graph. All `StateSnapshot`-s are saved in the checkpointer, and you can iterate over them.

---

The following cell creates a simple recursive graph that produces new states until the "value" reaches 4.

In [24]:
def condition(state: dict) -> str:
    if state["value"] >= 4:
        return "end"
    return "next"


def update(state: dict) -> dict:
    return {"value": state["value"] + 1}


graph = (
    StateGraph(dict)
    .add_node("update", update)
    .add_edge(START, "update")
    .add_conditional_edges(
        source="update",
        path=condition,
        path_map={"end": END, "next": "update"}
    )
    .compile(checkpointer=InMemorySaver())
)

As expected, the invocation of the graph returns a value of 4.

In [25]:
config = {"configurable": {"thread_id": 1}}
graph.invoke(dict(value=0), config=config)

{'value': 4}

The following  cell iterates through the history of the state updates and displays their values.

In [26]:
for state_check_point in graph.get_state_history(config):
    print(state_check_point.values)

{'value': 4}
{'value': 3}
{'value': 2}
{'value': 1}
{'value': 0}
None
